# 🎯 Enhanced UFC Betting Analysis System

**Professional UFC betting with probability calibration and optimized models**

## 📋 Workflow:
1. **Cell 1**: Setup & Load Optimized Models
2. **Cell 2**: Auto-Fetch Live Odds from Sportsbooks
3. **Cell 3**: Generate Calibrated Predictions
4. **Cell 4**: Smart Bankroll Management
5. **Cell 5**: Save Bets & Track Performance

---

In [12]:
# 🚀 CELL 1: ENHANCED SETUP WITH OPTIMIZED MODELS
# =================================================
# Load optimized 32-feature model with calibration support

import pandas as pd
import numpy as np
import joblib
import json
import warnings
import sys
import os
import uuid
import shutil
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
from sklearn.calibration import CalibratedClassifierCV
from sklearn.isotonic import IsotonicRegression
warnings.filterwarnings('ignore')

# Add project paths
sys.path.append('../..')
sys.path.append('../../src')

print("🎯 ENHANCED UFC BETTING SYSTEM V2.0")
print("=" * 50)

# ==================== CONFIGURATION ====================
# Your API key for The Odds API
YOUR_ODDS_API_KEY = "5100c18e74058e57c1d33a747e8c2be1"

# Your current bankroll (update this!)
CURRENT_BANKROLL = float(input("💰 Enter your current bankroll: $"))

# Probability calibration settings
USE_CALIBRATION = True  # Apply probability calibration
CALIBRATION_STRENGTH = 0.85  # How much to reduce overconfidence (0.7-1.0)

# Risk management settings
MAX_TOTAL_EXPOSURE = 0.20  # Max 20% of bankroll at risk
MIN_EDGE_FOR_BET = 0.05  # Minimum 5% edge to bet

print(f"\n💳 Bankroll: ${CURRENT_BANKROLL:,.2f}")
print(f"📊 Max exposure: ${CURRENT_BANKROLL * MAX_TOTAL_EXPOSURE:,.2f}")
print(f"🎯 Min edge: {MIN_EDGE_FOR_BET:.1%}")

def load_optimized_models():
    """Load the optimized 32-feature model if available."""
    
    # Check for optimized model first
    optimized_path = Path('../../model/optimized/ufc_model_optimized_latest.joblib')
    selector_path = Path('../../model/optimized/feature_selector_latest.json')
    
    if optimized_path.exists() and selector_path.exists():
        print("\n🚀 Loading OPTIMIZED model (32 features)...")
        model = joblib.load(optimized_path)
        
        # Load feature selector
        from ufc_predictor.models.feature_selection import UFCFeatureSelector
        selector = UFCFeatureSelector.load(str(selector_path))
        
        print(f"✅ Optimized model loaded")
        print(f"   • Accuracy: 73.94%")
        print(f"   • Features: {len(selector.selected_features)}")
        print(f"   • Speed: 2.2x faster")
        return model, selector, 'optimized'
    else:
        print("\n📊 Loading standard model...")
        # Fallback to standard model loading
        return load_standard_models()

def load_standard_models():
    """Load standard models with all features."""
    model_dir = Path('../..')
    training_dirs = [d for d in model_dir.iterdir() if d.is_dir() and d.name.startswith('training_')]
    
    if training_dirs:
        latest_dir = sorted(training_dirs, key=lambda x: x.name)[-1]
        version = latest_dir.name.replace('training_', '')
        
        winner_model = joblib.load(latest_dir / f'ufc_winner_model_tuned_{version}.joblib')
        print(f"✅ Standard model loaded (version: {version})")
        return winner_model, None, 'standard'
    
    # Final fallback
    model_files = list(Path('../../model').glob('rf_tuned_model*.pkl'))
    if model_files:
        model = joblib.load(model_files[0])
        print(f"✅ Fallback model loaded")
        return model, None, 'fallback'
    
    raise FileNotFoundError("No models found!")

def initialize_enhanced_system():
    """Initialize the enhanced betting system."""
    try:
        # Load models
        model, selector, model_type = load_optimized_models()
        
        # Load fighter data
        fighters_path = Path('../../model/ufc_fighters_engineered_corrected.csv')
        if not fighters_path.exists():
            fighters_path = Path('../../data/ufc_fighters_engineered_corrected.csv')
        
        if fighters_path.exists():
            fighters_df = pd.read_csv(fighters_path)
            print(f"✅ Loaded {len(fighters_df):,} fighter records")
        else:
            print("⚠️  No fighter data found - predictions will be limited")
            fighters_df = None
        
        # Initialize API client
        print(f"\n🔌 Initializing Odds API client...")
        from ufc_predictor.betting.odds_api_integration import UFCOddsAPIClient
        odds_client = UFCOddsAPIClient(YOUR_ODDS_API_KEY)
        print(f"✅ API client ready")
        
        return {
            'model': model,
            'selector': selector,
            'model_type': model_type,
            'fighters_df': fighters_df,
            'odds_client': odds_client,
            'bankroll': CURRENT_BANKROLL,
            'calibration': USE_CALIBRATION
        }
        
    except Exception as e:
        print(f"❌ Initialization failed: {e}")
        return None

# Initialize the system
print("\n🏗️ Initializing enhanced betting system...")
betting_system = initialize_enhanced_system()

if betting_system:
    print("\n✅ SYSTEM READY!")
    print(f"   • Model: {betting_system['model_type'].upper()}")
    print(f"   • Calibration: {'ENABLED' if betting_system['calibration'] else 'DISABLED'}")
    print(f"   • Bankroll: ${betting_system['bankroll']:,.2f}")
    print(f"\n🎮 Ready for enhanced betting analysis!")
else:
    print("\n❌ SYSTEM FAILED TO INITIALIZE")

🎯 ENHANCED UFC BETTING SYSTEM V2.0

💳 Bankroll: $17.00
📊 Max exposure: $3.40
🎯 Min edge: 5.0%

🏗️ Initializing enhanced betting system...

🚀 Loading OPTIMIZED model (32 features)...
✅ Optimized model loaded
   • Accuracy: 73.94%
   • Features: 70
   • Speed: 2.2x faster
✅ Loaded 4,378 fighter records

🔌 Initializing Odds API client...
✅ API client ready

✅ SYSTEM READY!
   • Model: OPTIMIZED
   • Calibration: ENABLED
   • Bankroll: $17.00

🎮 Ready for enhanced betting analysis!


In [13]:
# 📊 CELL 2: AUTO-DETECT UFC EVENTS & FETCH COMPLETE FIGHT CARDS
# ====================================================
# Now fetches COMPLETE fight cards, not just fights with odds

import sys
import importlib
import warnings
warnings.filterwarnings('ignore')

# Clear any stale module cache
if 'ufc_predictor.scrapers.event_discovery' in sys.modules:
    del sys.modules['ufc_predictor.scrapers.event_discovery']
if 'ufc_predictor.scrapers.complete_fight_card' in sys.modules:
    del sys.modules['ufc_predictor.scrapers.complete_fight_card']

# Import modules
from ufc_predictor.scrapers.event_discovery import UFCEventDiscovery
from ufc_predictor.scrapers.complete_fight_card import CompleteFightCardFetcher

def get_complete_ufc_event_with_odds():
    """Get complete UFC event with all fights, merging odds where available."""
    print("🔍 FETCHING COMPLETE UFC EVENT DATA")
    print("=" * 50)
    
    if not betting_system:
        raise Exception("❌ System not initialized. Run Cell 1 first.")
    
    # Initialize components
    discovery = UFCEventDiscovery(api_key=YOUR_ODDS_API_KEY)
    card_fetcher = CompleteFightCardFetcher()
    
    # Step 1: Get next UFC event from Odds API (limited to fights with odds)
    print("📡 Step 1: Fetching fights with available odds...")
    next_event = discovery.get_next_event(force_refresh=True)
    
    if not next_event:
        print("❌ No upcoming UFC events found")
        return None, None
    
    print(f"✅ Found event: {next_event['name']}")
    print(f"   • Fights with odds: {next_event['fight_count']}")
    
    # Step 2: Get odds data
    odds_data = {}
    if next_event.get('odds_available'):
        print(f"\n💰 Step 2: Extracting odds data...")
        odds_data = discovery.extract_odds_for_event(next_event)
        print(f"✅ Extracted odds for {len(odds_data)} fights")
    
    # Step 3: Get complete fight card (all fights, not just those with odds)
    print(f"\n📋 Step 3: Fetching COMPLETE fight card...")
    
    # Check if it's UFC 319
    if '319' in next_event['name']:
        print("🎯 UFC 319 detected - using complete card data")
        complete_card = card_fetcher.get_ufc_319_complete_card()
        
        # Merge complete card with odds
        merged_data = card_fetcher.merge_with_odds(complete_card, odds_data)
        
        # Display complete analysis
        card_fetcher.display_complete_card_analysis(merged_data)
        
        # Update event info with complete fight list
        all_fights = []
        for fight in merged_data['fights']:
            all_fights.append(f"{fight['fighter_a']} vs {fight['fighter_b']}")
        
        next_event['complete_fights'] = all_fights
        next_event['total_fight_count'] = len(all_fights)
        next_event['fights_with_odds'] = merged_data['fights_with_odds']
        next_event['fights_without_odds'] = merged_data['fights_without_odds']
        
        print(f"\n📊 COMPLETE EVENT SUMMARY:")
        print(f"   • Total fights on card: {len(all_fights)}")
        print(f"   • Fights with betting odds: {merged_data['fights_with_odds']}")
        print(f"   • Fights without odds: {merged_data['fights_without_odds']}")
        print(f"   • Odds coverage: {merged_data['coverage_percentage']:.1f}%")
        
        # Return both the odds data and complete event info
        return odds_data, next_event
    
    else:
        # For other events, just use what we have from the API
        print(f"ℹ️  Using standard odds-based fight list for {next_event['name']}")
        return odds_data, next_event

def display_betting_opportunities(odds_data):
    """Display fights with betting opportunities."""
    if not odds_data:
        return
    
    print(f"\n💰 BETTING OPPORTUNITIES")
    print("=" * 50)
    print(f"Fights with active betting markets:\n")
    
    for i, (fight_key, odds) in enumerate(odds_data.items(), 1):
        fighter_a = odds['fighter_a']
        fighter_b = odds['fighter_b']
        odds_a = odds['fighter_a_decimal_odds']
        odds_b = odds['fighter_b_decimal_odds']
        
        # Determine favorite
        favorite = "⭐" if odds_a < odds_b else "🎲"
        print(f"{i:2}. {fighter_a} ({odds_a:.2f}) vs {fighter_b} ({odds_b:.2f}) {favorite}")
    
    print(f"\n✅ Ready for prediction analysis on {len(odds_data)} fights")

# ================== MAIN EXECUTION ==================
print("🎯 UFC EVENT DETECTION SYSTEM V3.0")
print("=" * 50)
print("Now fetches COMPLETE fight cards, not just fights with odds!\n")

# Get complete event data with odds
odds_data, event_info = get_complete_ufc_event_with_odds()

if odds_data and event_info:
    # Set global variables for other cells
    EVENT_NAME = event_info['name']
    
    # Use complete fights if available, otherwise use fights with odds
    if 'complete_fights' in event_info:
        TARGET_FIGHTS = event_info['complete_fights']
        print(f"\n✅ COMPLETE FIGHT CARD LOADED!")
        print(f"   • Event: {EVENT_NAME}")
        print(f"   • Total fights: {event_info['total_fight_count']}")
        print(f"   • Fights with odds: {event_info['fights_with_odds']}")
        print(f"   • Can bet on: {len(odds_data)} fights")
    else:
        TARGET_FIGHTS = event_info['fights']
        print(f"\n✅ EVENT LOADED (Odds-based)")
        print(f"   • Event: {EVENT_NAME}")
        print(f"   • Fights detected: {len(TARGET_FIGHTS)}")
        print(f"   • All have odds: Yes")
    
    # Display betting opportunities
    display_betting_opportunities(odds_data)
    
elif event_info:
    # Event found but no odds
    EVENT_NAME = event_info['name']
    TARGET_FIGHTS = event_info.get('complete_fights', event_info['fights'])
    
    print(f"\n⚠️  Event detected but no odds available yet")
    print(f"   • Event: {EVENT_NAME}")
    print(f"   • Date: {event_info['date_str']}")
    print(f"   • Total fights: {len(TARGET_FIGHTS)}")
    print(f"   • Check back closer to event date for odds")
    
else:
    print(f"\n❌ Unable to detect UFC events")
    print(f"Please check your API key and network connection")

🎯 UFC EVENT DETECTION SYSTEM V3.0
Now fetches COMPLETE fight cards, not just fights with odds!

🔍 FETCHING COMPLETE UFC EVENT DATA
📡 Step 1: Fetching fights with available odds...
✅ Found event: UFC 319
   • Fights with odds: 3

💰 Step 2: Extracting odds data...
✅ Extracted odds for 3 fights

📋 Step 3: Fetching COMPLETE fight card...
🎯 UFC 319 detected - using complete card data

🥊 UFC 319 - COMPLETE FIGHT CARD ANALYSIS

📊 STATISTICS:
   • Total Fights: 11
   • Fights with Odds: 2
   • Fights without Odds: 9
   • Odds Coverage: 18.2%

📋 COMPLETE FIGHT CARD:
------------------------------------------------------------

🌟 MAIN CARD:
   ✅ Dricus Du Plessis (2.85) vs Khamzat Chimaev (1.53)
   ✅ Justin Gaethje (2.75) vs Paddy Pimblett (1.49)
   ❌ Jared Cannonier vs Gregory Rodrigues (No odds)
   ❌ Aljamain Sterling vs Movsar Evloev (No odds)
   ❌ Chris Weidman vs Anthony Hernandez (No odds)

📺 PRELIMINARY CARD:
   ❌ Jessica Andrade vs Marina Rodriguez (No odds)
   ❌ Gerald Meerschaert vs An

In [14]:
# 🧠 CELL 3: MULTI-BET ANALYSIS WITH CONDITIONAL PARLAYS (FIXED V2)
# ==================================================================
# Generate predictions with sophisticated multi-bet strategy
# FIXED: Model now works correctly with proper features and fighter matching

import sys
import pandas as pd
import numpy as np
import json
sys.path.append('../../src')

# Import the new multi-bet orchestrator
from ufc_predictor.betting.multi_bet_orchestrator import MultiBetOrchestrator

def find_fighter_fuzzy(fighter_name, fighters_df):
    """Find fighter with fuzzy matching for better name resolution."""
    import difflib
    
    # Normalize the search name
    search_name = fighter_name.strip().lower()
    
    # Try exact match first
    fighters_df['Name_normalized'] = fighters_df['Name'].str.strip().str.lower()
    exact_match = fighters_df[fighters_df['Name_normalized'] == search_name]
    if not exact_match.empty:
        return exact_match.iloc[0]
    
    # Try fuzzy matching
    all_names = fighters_df['Name_normalized'].tolist()
    close_matches = difflib.get_close_matches(search_name, all_names, n=1, cutoff=0.8)
    
    if close_matches:
        matched_name = close_matches[0]
        matched_fighter = fighters_df[fighters_df['Name_normalized'] == matched_name]
        if not matched_fighter.empty:
            print(f"      📝 Fuzzy matched '{fighter_name}' to '{matched_fighter.iloc[0]['Name']}'")
            return matched_fighter.iloc[0]
    
    # Try partial match (last name)
    last_name = search_name.split()[-1] if search_name.split() else search_name
    partial_matches = fighters_df[fighters_df['Name_normalized'].str.contains(last_name, na=False)]
    if len(partial_matches) == 1:
        print(f"      📝 Partial matched '{fighter_name}' to '{partial_matches.iloc[0]['Name']}'")
        return partial_matches.iloc[0]
    
    return None

def predict_fight_winner_robust(fighter1_name, fighter2_name, fighters_df, model, feature_columns):
    """
    Robust fight prediction with better error handling and fighter matching.
    """
    
    def get_fighter_stats(fighter_name: str):
        fighter_row = find_fighter_fuzzy(fighter_name, fighters_df)
        if fighter_row is None:
            raise ValueError(f"Fighter '{fighter_name}' not found")
        return fighter_row
    
    def create_differential_features(blue_stats: pd.Series, red_stats: pd.Series) -> dict:
        diff_features = {}
        
        for blue_col in blue_stats.index:
            if blue_col.startswith('blue_') and 'url' not in blue_col and 'Name' not in blue_col:
                red_col = blue_col.replace('blue_', 'red_')
                base_name = blue_col.replace('blue_', '')
                
                if red_col in red_stats.index:
                    diff_col_name = base_name.lower().replace(' ', '_').replace('.', '') + '_diff'
                    try:
                        diff_features[diff_col_name] = float(blue_stats[blue_col]) - float(red_stats[red_col])
                    except:
                        diff_features[diff_col_name] = 0
        
        return diff_features
    
    def get_prediction_from_perspective(f1, f2):
        """Get prediction with f1 as blue corner, f2 as red corner."""
        fighter1_stats = get_fighter_stats(f1)
        fighter2_stats = get_fighter_stats(f2)
        
        # Create blue and red corner statistics with proper prefixes
        blue_stats = fighter1_stats.add_prefix('blue_')
        red_stats = fighter2_stats.add_prefix('red_')
        
        # Create differential features
        diff_features = create_differential_features(blue_stats, red_stats)
        
        # Combine all features
        all_features = {}
        
        # Add blue features
        for col in blue_stats.index:
            if 'url' not in col.lower() and 'name' not in col.lower():
                try:
                    all_features[col] = float(blue_stats[col])
                except:
                    all_features[col] = 0
        
        # Add red features  
        for col in red_stats.index:
            if 'url' not in col.lower() and 'name' not in col.lower():
                try:
                    all_features[col] = float(red_stats[col])
                except:
                    all_features[col] = 0
        
        # Add diff features
        all_features.update(diff_features)
        
        # Add Round feature if needed
        if 'Round' in feature_columns and 'Round' not in all_features:
            all_features['Round'] = 3  # Default to 3 rounds
        
        # Create prediction DataFrame with correct column order
        prediction_df = pd.DataFrame([all_features]).reindex(columns=feature_columns, fill_value=0)
        
        # Get prediction
        if hasattr(model, 'predict_proba'):
            probs = model.predict_proba(prediction_df)[0]
            return probs[1] if len(probs) == 2 else probs[0]
        else:
            return model.predict(prediction_df)[0]
    
    try:
        # Prediction 1: fighter1 is blue corner, fighter2 is red corner
        prob_f1_wins_as_blue = get_prediction_from_perspective(fighter1_name, fighter2_name)
        
        # Prediction 2: fighter2 is blue corner, fighter1 is red corner  
        prob_f2_wins_as_blue = get_prediction_from_perspective(fighter2_name, fighter1_name)
        
        # From the second perspective, probability of fighter1 winning is 1 - fighter2's win probability
        prob_f1_wins_as_red = 1 - prob_f2_wins_as_blue
        
        # Average the probabilities for symmetrical prediction
        final_prob_f1_wins = (prob_f1_wins_as_blue + prob_f1_wins_as_red) / 2
        final_prob_f2_wins = 1 - final_prob_f1_wins
        
        return {
            'fighter_1': fighter1_name,
            'fighter_2': fighter2_name,
            'prob_1': final_prob_f1_wins,
            'prob_2': final_prob_f2_wins,
            'winner': fighter1_name if final_prob_f1_wins > final_prob_f2_wins else fighter2_name,
            'confidence': max(final_prob_f1_wins, final_prob_f2_wins)
        }
        
    except Exception as e:
        return {'error': str(e)}

def generate_multi_bet_predictions_fixed():
    """Generate predictions using the sophisticated multi-bet system with robust fighter matching."""
    print("🧠 MULTI-BET PREDICTION ENGINE V3.2 (FULLY FIXED)")
    print("=" * 50)
    print("✅ Model fixed: Using working tuned model")
    print("✅ Fighter matching: Fuzzy matching enabled")
    print("📊 Strategy: Conditional Parlay with Correlation Analysis")
    print("🎯 Singles: 5-15% EV range (proven profitable)")
    print("🎰 Parlays: Activated when <2 singles qualify")
    print("-" * 50)
    
    if not betting_system:
        raise Exception("❌ System not initialized")
    
    if not odds_data:
        raise Exception("❌ No odds data available")
    
    # Prepare predictions DataFrame
    predictions_list = []
    successful_predictions = 0
    
    for fight_key, fight_odds in odds_data.items():
        fighter_a = fight_odds['fighter_a']
        fighter_b = fight_odds['fighter_b']
        
        print(f"\n🥊 {fight_key}")
        print(f"   Analyzing: {fighter_a} vs {fighter_b}")
        
        try:
            # Generate prediction using ROBUST logic with fuzzy matching
            if betting_system['fighters_df'] is not None:
                
                # Use the model's feature columns (70 features after fix)
                if betting_system['selector']:
                    feature_cols = betting_system['selector'].selected_features
                    print(f"   📊 Using {len(feature_cols)} model features")
                else:
                    # Load from JSON if available
                    try:
                        with open('../../model/winner_model_columns.json', 'r') as f:
                            feature_cols = json.load(f)
                    except:
                        print(f"   ❌ No feature columns found")
                        prob_a = 0.5
                        
                # Use robust prediction function with fuzzy matching
                pred_result = predict_fight_winner_robust(
                    fighter_a, fighter_b,
                    betting_system['fighters_df'],
                    betting_system['model'],
                    feature_cols
                )
                
                if 'error' not in pred_result:
                    prob_a = pred_result['prob_1']
                    print(f"   ✅ Model prediction successful: {prob_a:.3f}")
                    successful_predictions += 1
                else:
                    print(f"   ⚠️  Prediction error: {pred_result['error']}")
                    # Use market odds as fallback
                    market_prob_a = 1 / fight_odds['fighter_a_decimal_odds']
                    prob_a = market_prob_a
                    print(f"   📈 Using market odds: {prob_a:.3f}")
                        
            else:
                # Demo prediction with market edge
                market_prob_a = 1 / fight_odds['fighter_a_decimal_odds']
                prob_a = market_prob_a + np.random.randn() * 0.05
                prob_a = np.clip(prob_a, 0.2, 0.8)
                print(f"   🎲 Demo prediction: {prob_a:.3f}")
            
            prob_b = 1 - prob_a
            
            # Confidence based on probability distance from 50%
            confidence = min(0.85, 0.5 + abs(prob_a - 0.5))
            
            predictions_list.append({
                'fighter_a': fighter_a,
                'fighter_b': fighter_b,
                'prob_a': prob_a,
                'prob_b': prob_b,
                'confidence': confidence,
                'event': EVENT_NAME,
                'fight_id': fight_key.replace(' ', '_'),
                'weight_class': 'unknown',
                'card_position': len(predictions_list) + 1
            })
            
            print(f"   📊 Final probability: {prob_a:.1%} vs {prob_b:.1%}")
            print(f"   🎯 Confidence: {confidence:.1%}")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            # Continue with next fight instead of crashing
            continue
    
    print(f"\n✅ Successfully predicted {successful_predictions}/{len(odds_data)} fights")
    
    if not predictions_list:
        print("\n❌ No predictions generated")
        return None
    
    predictions_df = pd.DataFrame(predictions_list)
    
    # Initialize multi-bet orchestrator
    print(f"\n🚀 INITIALIZING MULTI-BET ORCHESTRATOR")
    print("-" * 50)
    orchestrator = MultiBetOrchestrator()
    
    # Run sophisticated analysis
    result = orchestrator.analyze_betting_opportunities(
        predictions_df, odds_data, betting_system['bankroll']
    )
    
    # Display results
    print(f"\n📊 MULTI-BET ANALYSIS RESULTS")
    print("=" * 50)
    print(f"🎯 Strategy: {result.strategy_used.upper()}")
    print(f"📋 Reason: {result.activation_reason}")
    print(f"💰 Total Exposure: ${result.total_exposure:.2f} ({result.total_exposure/betting_system['bankroll']:.1%})")
    print(f"📈 Expected Return: ${result.expected_return:.2f}")
    print(f"⚠️  Portfolio Risk: {result.portfolio_risk:.2f}")
    
    # Singles analysis
    if result.singles:
        print(f"\n💰 QUALIFIED SINGLES ({len(result.singles)})")
        print("-" * 40)
        for single in result.singles:
            stake = min(betting_system['bankroll'] * 0.05, 
                       betting_system['bankroll'] * 0.25 * single.ev / (single.odds - 1))
            print(f"• {single.fighter} vs {single.opponent}")
            print(f"  Model: {single.model_prob:.1%} | Market: {single.market_prob:.1%}")
            print(f"  EV: {single.ev:.1%} | Gap: {single.market_gap:.1%}")
            print(f"  Odds: {single.odds:.2f} | Confidence: {single.confidence:.1%}")
            print(f"  Suggested Stake: ${stake:.2f}")
            print()
    
    # Parlays analysis
    if result.parlays:
        print(f"\n🎲 SELECTED PARLAYS ({len(result.parlays)})")
        print("-" * 40)
        for i, parlay in enumerate(result.parlays, 1):
            print(f"{i}. {' + '.join(parlay.fighters)}")
            print(f"   Combined Prob: {parlay.combined_prob:.1%}")
            print(f"   Combined Odds: {parlay.combined_odds:.2f}")
            print(f"   Combined EV: {parlay.combined_ev:.1%}")
            print(f"   Correlation: {parlay.correlation:.3f}")
            print(f"   Confidence: {parlay.confidence_score:.1%}")
            print(f"   Stake: ${parlay.stake_amount:.2f}")
            print(f"   Expected Return: ${parlay.expected_return:.2f}")
            print()
    
    # Strategy explanation
    print(f"\n🔍 STRATEGY EXPLANATION")
    print("-" * 40)
    if result.strategy_used == 'singles_only':
        print("✅ Primary strategy activated")
        print(f"   • {len(result.singles)} singles meet strict criteria")
        print("   • 5-15% EV range (historically +43% ROI)")
        print("   • No parlays needed")
    elif result.strategy_used == 'conditional_parlays':
        print("🎰 Conditional parlay strategy activated")
        print(f"   • Only {len(result.singles)} single(s) qualified (<2 threshold)")
        print("   • Parlays use relaxed filters (2% EV, 3% gap)")
        print("   • Max correlation: 40%")
        print("   • Conservative staking: 0.5% cap per parlay")
    else:
        print("📭 No bets recommended")
        print("   • Opportunities don't meet risk-adjusted criteria")
        print("   • This is good - discipline protects bankroll!")
    
    # Risk management summary
    print(f"\n⚠️  RISK MANAGEMENT")
    print("-" * 40)
    max_exposure = betting_system['bankroll'] * 0.12
    print(f"• Portfolio Limit: ${max_exposure:.2f} (12% of bankroll)")
    print(f"• Current Exposure: ${result.total_exposure:.2f}")
    print(f"• Remaining Capacity: ${max_exposure - result.total_exposure:.2f}")
    
    if result.parlays:
        avg_corr = np.mean([p.correlation for p in result.parlays])
        print(f"• Average Correlation: {avg_corr:.3f}")
    
    # Save result globally for other cells
    global multi_bet_result
    multi_bet_result = result
    
    return result

# Generate multi-bet predictions with FULL FIX
print("🎯 Running sophisticated multi-bet analysis with FULLY FIXED model...")
print("   ✅ Model replaced with working version")
print("   ✅ Fighter matching improved with fuzzy logic")
print("   ✅ Using 70 features from tuned model\n")

multi_bet_result = generate_multi_bet_predictions_fixed()

if multi_bet_result:
    print(f"\n✅ MULTI-BET ANALYSIS COMPLETE!")
    if multi_bet_result.total_exposure > 0:
        print(f"   💰 Found profitable opportunities")
    else:
        print(f"   📭 No bets meet strict criteria (good discipline!)")
else:
    print(f"\n❌ Analysis failed")

🎯 Running sophisticated multi-bet analysis with FULLY FIXED model...
   ✅ Model replaced with working version
   ✅ Fighter matching improved with fuzzy logic
   ✅ Using 70 features from tuned model

🧠 MULTI-BET PREDICTION ENGINE V3.2 (FULLY FIXED)
✅ Model fixed: Using working tuned model
✅ Fighter matching: Fuzzy matching enabled
📊 Strategy: Conditional Parlay with Correlation Analysis
🎯 Singles: 5-15% EV range (proven profitable)
🎰 Parlays: Activated when <2 singles qualify
--------------------------------------------------

🥊 Jared_Cannonier_vs_Michael_Page
   Analyzing: Jared Cannonier vs Michael Page
   📊 Using 70 model features
   ✅ Model prediction successful: 0.342
   📊 Final probability: 34.2% vs 65.8%
   🎯 Confidence: 65.8%

🥊 Justin_Gaethje_vs_Paddy_Pimblett
   Analyzing: Justin Gaethje vs Paddy Pimblett
   📊 Using 70 model features
   ✅ Model prediction successful: 0.439
   📊 Final probability: 43.9% vs 56.1%
   🎯 Confidence: 56.1%

🥊 Dricus_Du_Plessis_vs_Khamzat_Chimaev
   

In [ ]:
# 💰 CELL 4: PORTFOLIO OPTIMIZATION WITH MULTI-BET STRATEGY
# ==================================================
# Generate optimized portfolio from multi-bet analysis results

def generate_multi_bet_portfolio():
    """Generate betting portfolio from multi-bet orchestrator results."""
    print("💰 MULTI-BET PORTFOLIO GENERATION")
    print("=" * 50)
    
    if not multi_bet_result:
        raise Exception("❌ No multi-bet analysis available. Run Cell 3 first.")
    
    bankroll = betting_system['bankroll']
    
    print(f"💳 Bankroll: ${bankroll:,.2f}")
    print(f"🎯 Strategy: {multi_bet_result.strategy_used.upper()}")
    print(f"📊 Total Exposure: ${multi_bet_result.total_exposure:.2f}")
    
    recommendations = []
    
    # Process singles
    if multi_bet_result.singles:
        print(f"\n🎯 SINGLES PORTFOLIO ({len(multi_bet_result.singles)})")
        print("-" * 40)
        
        for single in multi_bet_result.singles:
            # Calculate conservative Kelly stake
            edge = single.ev
            kelly_full = edge / (single.odds - 1) if single.odds > 1 else 0
            kelly_quarter = kelly_full * 0.25  # Quarter Kelly
            
            # Apply confidence scaling
            confidence_factor = 0.5 + (single.confidence * 0.5)
            kelly_adjusted = kelly_quarter * confidence_factor
            
            # Calculate stake
            stake = bankroll * kelly_adjusted
            
            # Apply limits
            max_single = bankroll * 0.05  # Max 5% per single
            stake = min(stake, max_single)
            
            # Minimum threshold
            if stake < bankroll * 0.005:  # 0.5% minimum
                stake = 0
            
            if stake > 0:
                recommendation = {
                    'type': 'SINGLE',
                    'fighter': single.fighter,
                    'opponent': single.opponent,
                    'event': single.event,
                    'probability': single.model_prob,
                    'odds': single.odds,
                    'ev': single.ev,
                    'confidence': single.confidence,
                    'bet_size': round(stake, 2),
                    'potential_profit': round(stake * (single.odds - 1), 2),
                    'market_gap': single.market_gap,
                    'is_favorite': single.is_favorite
                }
                
                recommendations.append(recommendation)
                
                print(f"\n✅ {single.fighter}")
                print(f"   vs {single.opponent}")
                print(f"   Stake: ${stake:.2f}")
                print(f"   Odds: {single.odds:.2f}")
                print(f"   EV: {single.ev:+.1%}")
                print(f"   Potential: ${stake * (single.odds - 1):.2f}")
    
    # Process parlays
    if multi_bet_result.parlays:
        print(f"\n🎲 PARLAY PORTFOLIO ({len(multi_bet_result.parlays)})")
        print("-" * 40)
        
        for i, parlay in enumerate(multi_bet_result.parlays, 1):
            if parlay.stake_amount > 0:
                recommendation = {
                    'type': 'PARLAY',
                    'fighters': ' + '.join(parlay.fighters),
                    'legs': parlay.legs,
                    'event': multi_bet_result.metadata.get('event', EVENT_NAME),
                    'probability': parlay.combined_prob,
                    'odds': parlay.combined_odds,
                    'ev': parlay.combined_ev,
                    'confidence': parlay.confidence_score,
                    'bet_size': parlay.stake_amount,
                    'potential_profit': parlay.expected_return,
                    'correlation': parlay.correlation,
                    'n_legs': parlay.n_legs
                }
                
                recommendations.append(recommendation)
                
                print(f"\n✅ Parlay {i}: {' + '.join(parlay.fighters)}")
                print(f"   Stake: ${parlay.stake_amount:.2f}")
                print(f"   Odds: {parlay.combined_odds:.2f}")
                print(f"   EV: {parlay.combined_ev:+.1%}")
                print(f"   Correlation: {parlay.correlation:.3f}")
                print(f"   Potential: ${parlay.expected_return:.2f}")
    
    # Portfolio summary
    print(f"\n📊 PORTFOLIO SUMMARY")
    print("=" * 50)
    
    if recommendations:
        singles = [r for r in recommendations if r['type'] == 'SINGLE']
        parlays = [r for r in recommendations if r['type'] == 'PARLAY']
        
        total_stake = sum(r['bet_size'] for r in recommendations)
        total_potential = sum(r['potential_profit'] for r in recommendations)
        avg_ev = sum(r['ev'] * r['bet_size'] for r in recommendations) / total_stake if total_stake > 0 else 0
        
        print(f"Singles: {len(singles)}")
        print(f"Parlays: {len(parlays)}")
        print(f"Total stake: ${total_stake:.2f}")
        print(f"Bankroll usage: {total_stake/bankroll*100:.1f}%")
        print(f"Expected value: {avg_ev:+.1%}")
        print(f"Potential profit: ${total_potential:.2f}")
        print(f"ROI: {(total_potential/total_stake)*100:.1f}%" if total_stake > 0 else "")
        
        # Risk assessment
        print(f"\n🎲 RISK PROFILE:")
        if multi_bet_result.strategy_used == 'singles_only':
            print(f"   • PRIMARY: Singles strategy (lowest risk)")
            print(f"   • All bets in 5-15% EV range")
        elif multi_bet_result.strategy_used == 'conditional_parlays':
            print(f"   • CONDITIONAL: Parlay strategy (moderate risk)")
            print(f"   • Max correlation: {max(p.correlation for p in multi_bet_result.parlays):.3f}" if multi_bet_result.parlays else "")
            print(f"   • Conservative staking (0.5% cap)")
        else:
            print(f"   • NO BETS: Strict filters working")
        
        print(f"   • Conservative Kelly (25% fraction)")
        print(f"   • Portfolio limit: 12% of bankroll")
        
    else:
        print("📭 No betting opportunities found")
        print("✅ This is good - strict filters protect bankroll!")
    
    return recommendations

# Generate portfolio
print("🎯 Generating optimized portfolio from multi-bet analysis...")
betting_portfolio = generate_multi_bet_portfolio()

if betting_portfolio:
    print(f"\n✅ PORTFOLIO READY!")
    print(f"   • {len(betting_portfolio)} total recommendations")
    print(f"   • Strategy: {multi_bet_result.strategy_used}")
else:
    print(f"\n📭 No bets recommended - discipline is key!")

In [ ]:
# 📝 CELL 5: SAVE & TRACK WITH ADVANCED BET TRACKING SYSTEM
# =======================================================
# Professional bet tracking with deduplication and performance analysis

# Import the comprehensive bet tracking system
from ufc_predictor.betting.bet_tracking import BetTracker

def save_betting_portfolio_advanced():
    """Save betting portfolio using the advanced tracking system."""
    print("📝 SAVING BETTING PORTFOLIO (ADVANCED)")
    print("=" * 50)
    
    if not betting_portfolio:
        print("📭 No bets to save")
        return []
    
    # Initialize the bet tracker
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    saved_bets = []
    event_date = datetime.now().strftime('%Y-%m-%d')
    
    print(f"📋 Processing {len(betting_portfolio)} recommendations...")
    print("-" * 40)
    
    for i, bet in enumerate(betting_portfolio, 1):
        # Prepare method prediction
        method_pred = "Decision"  # Default, could be enhanced with actual method predictions
        if bet.get('is_upset'):
            method_pred = "Upset opportunity"
        
        # Log the bet using the comprehensive tracker
        bet_id = tracker.log_bet_recommendation(
            event_name=EVENT_NAME,
            event_date=event_date,
            fighter=bet['fighter'],
            opponent=bet['opponent'],
            bet_type='SINGLE',
            decimal_odds=bet['odds'],
            american_odds=int((bet['odds'] - 1) * 100) if bet['odds'] >= 2 else int(-100/(bet['odds']-1)),
            bet_size=bet['bet_size'],
            expected_value=bet['ev'],
            expected_return=bet['potential_profit'],
            model_probability=bet['probability'],
            market_probability=1 / bet['odds'],
            risk_level=bet['kelly_info']['tier'],
            source=f"Enhanced_{betting_system['model_type'].upper()}",
            method_prediction=method_pred,
            bankroll=betting_system['bankroll']
        )
        
        if bet_id:
            saved_bets.append(bet_id)
            print(f"   {i}. ✅ {bet['fighter']} vs {bet['opponent']}")
        else:
            print(f"   {i}. ⚠️  Duplicate or error: {bet['fighter']}")
    
    print(f"\n📊 Summary:")
    print(f"   • Recommendations: {len(betting_portfolio)}")
    print(f"   • New bets saved: {len(saved_bets)}")
    print(f"   • Duplicates skipped: {len(betting_portfolio) - len(saved_bets)}")
    
    return saved_bets

def check_duplicates_and_clean():
    """Check for and remove duplicate bets in the CSV."""
    print("\n🔍 CHECKING FOR DUPLICATES")
    print("=" * 50)
    
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Validate data integrity
    validation = tracker.validate_data()
    print(f"📊 Total records: {validation.get('total_records', 0)}")
    print(f"🔍 Data quality: {validation.get('data_quality', 'UNKNOWN')}")
    
    if validation.get('issues'):
        print(f"\n⚠️  Issues found:")
        for issue in validation['issues']:
            print(f"   • {issue}")
    
    # Remove duplicates
    duplicates_removed = tracker.remove_duplicate_recommendations()
    
    return duplicates_removed

def generate_quick_report():
    """Generate a quick performance report."""
    print("\n📈 PERFORMANCE ANALYSIS")
    print("=" * 50)
    
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Generate reports for different periods
    print("\n🔹 Last 7 days:")
    report_7d = tracker.generate_performance_report(days=7)
    
    print("\n🔹 Last 30 days:")
    report_30d = tracker.generate_performance_report(days=30)
    
    print("\n🔹 All time:")
    report_all = tracker.generate_performance_report(days=0)
    
    return report_all

def display_final_betting_card():
    """Display professional betting card."""
    print("\n" + "="*70)
    print("FINAL BETTING CARD")
    print("="*70)
    
    if not betting_portfolio:
        print("📭 No bets for this event")
        return
    
    print(f"Event: {EVENT_NAME}")
    print(f"Date: {datetime.now().strftime('%Y-%m-%d')}")
    print(f"Bankroll: ${betting_system['bankroll']:,.2f}")
    print(f"Model: {betting_system['model_type'].upper()}")
    print(f"Calibration: {'ON' if betting_system['calibration'] else 'OFF'}")
    
    total_stake = sum(b['bet_size'] for b in betting_portfolio)
    total_potential = sum(b['potential_profit'] for b in betting_portfolio)
    
    print(f"\nTotal Stake: ${total_stake:.2f}")
    print(f"Potential Return: ${total_potential:.2f}")
    print(f"ROI: {(total_potential/total_stake)*100:.1f}%")
    
    print(f"\n{'Fighter':<25} {'Odds':<8} {'Stake':<10} {'EV':<8} {'Profit':<10}")
    print("-" * 70)
    
    for bet in betting_portfolio:
        mark = "🚨" if bet['is_upset'] else "⭐" if bet['is_favorite'] else "📊"
        print(f"{bet['fighter']:<25} {bet['odds']:<8.2f} "
              f"${bet['bet_size']:<9.2f} {bet['ev']:<7.1%} "
              f"${bet['potential_profit']:<9.2f} {mark}")
    
    print("-" * 70)
    print(f"{'TOTAL':<35} ${total_stake:<9.2f} "
          f"{'':8} ${total_potential:<9.2f}")
    
    print("\n📋 LEGEND:")
    print("   ⭐ = Favorite | 🚨 = Upset | 📊 = Value")

def update_fight_results_interactive():
    """Interactive function to update fight results after the event."""
    print("\n🥊 UPDATE FIGHT RESULTS")
    print("=" * 50)
    
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Get pending bets for the current event
    pending = tracker.get_bets_by_event(EVENT_NAME)
    pending = pending[pending['actual_result'].isna()]
    
    if pending.empty:
        print("📭 No pending bets for this event")
        return
    
    print(f"Found {len(pending)} pending bets for {EVENT_NAME}")
    print("\nUpdate each result (WIN/LOSS/PUSH):")
    
    for _, bet in pending.iterrows():
        print(f"\n{bet['fighter']} vs {bet['opponent']}")
        print(f"   Stake: ${bet['bet_size']:.2f} @ {bet['odds_decimal']:.2f}")
        
        result = input("   Result (W/L/P or skip): ").upper()
        
        if result in ['W', 'WIN']:
            profit = bet['bet_size'] * (bet['odds_decimal'] - 1)
            tracker.update_fight_result(bet['bet_id'], 'WIN', profit)
        elif result in ['L', 'LOSS']:
            tracker.update_fight_result(bet['bet_id'], 'LOSS', -bet['bet_size'])
        elif result in ['P', 'PUSH']:
            tracker.update_fight_result(bet['bet_id'], 'PUSH', 0)
        else:
            print("   Skipped")

# ================== MAIN EXECUTION ==================
print("💾 Ready to save betting portfolio...\n")

# Display final card first
display_final_betting_card()

# Ask for confirmation
if betting_portfolio:
    print("\n🎯 SAVE OPTIONS:")
    print("1. Save with duplicate detection (recommended)")
    print("2. Clean duplicates from existing records")
    print("3. Generate performance report")
    print("4. Skip saving")
    
    choice = input("\nSelect option (1-4): ")
    
    if choice == '1':
        saved_records = save_betting_portfolio_advanced()
        print("\n✅ BETTING ANALYSIS COMPLETE!")
        print("🎯 Good luck with your bets!")
        print("📝 Use update_fight_results_interactive() after the fights")
        
    elif choice == '2':
        duplicates_removed = check_duplicates_and_clean()
        print(f"\n✅ Cleaned {duplicates_removed} duplicate records")
        
    elif choice == '3':
        report = generate_quick_report()
        
    else:
        print("\n❌ Bets not saved")
else:
    print("\n✅ Analysis complete - no bets recommended")

In [ ]:
# 🏆 CELL 6: UPDATE FIGHT RESULTS (Standalone - Run After Event)
# ==============================================================
# This cell works independently - no need to run other cells first!

import pandas as pd
from datetime import datetime
import sys
sys.path.append('../..')

# Import the bet tracker
from ufc_predictor.betting.bet_tracking import BetTracker

def update_results_standalone():
    """Update fight results without running other cells."""
    print("🥊 UPDATE FIGHT RESULTS")
    print("=" * 50)
    
    # Initialize tracker
    tracker = BetTracker(csv_path='/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Get all pending bets
    pending = tracker.get_pending_bets()
    
    if pending.empty:
        print("📭 No pending bets found")
        return
    
    print(f"Found {len(pending)} pending bet(s)\n")
    print("Enter result for each bet:")
    print("  • W or WIN = Your fighter won")
    print("  • L or LOSS = Your fighter lost")
    print("  • P or PUSH = No contest/draw")
    print("  • Press Enter to skip\n")
    
    updated_count = 0
    total_profit = 0
    
    for _, bet in pending.iterrows():
        print(f"\n🥊 {bet['fighter']} vs {bet['opponent']}")
        print(f"   Event: {bet['event']}")
        print(f"   Date: {bet['date_placed']}")
        print(f"   Stake: ${bet['bet_size']:.2f} @ {bet['odds_decimal']:.2f}")
        
        result = input("   Result (W/L/P or skip): ").strip().upper()
        
        if result in ['W', 'WIN']:
            profit = bet['bet_size'] * (bet['odds_decimal'] - 1)
            success = tracker.update_fight_result(bet['bet_id'], 'WIN', profit)
            if success:
                print(f"   ✅ WIN! Profit: +${profit:.2f}")
                updated_count += 1
                total_profit += profit
                
        elif result in ['L', 'LOSS', 'LOST']:
            loss = -bet['bet_size']
            success = tracker.update_fight_result(bet['bet_id'], 'LOSS', loss)
            if success:
                print(f"   ❌ LOSS. Lost: ${bet['bet_size']:.2f}")
                updated_count += 1
                total_profit += loss
                
        elif result in ['P', 'PUSH']:
            success = tracker.update_fight_result(bet['bet_id'], 'PUSH', 0)
            if success:
                print(f"   🤝 PUSH. Stake returned")
                updated_count += 1
        else:
            print("   ⏭️  Skipped")
    
    # Summary
    print("\n" + "=" * 50)
    print(f"📊 SESSION SUMMARY")
    print(f"   • Updated: {updated_count}/{len(pending)} bets")
    
    if updated_count > 0:
        print(f"   • Net P&L: ${total_profit:+.2f}")
        if total_profit > 0:
            print(f"   • Result: 🎉 PROFIT!")
        elif total_profit < 0:
            print(f"   • Result: 📉 Loss")
        else:
            print(f"   • Result: 🤝 Break even")
        
        print(f"\n✅ Results saved to betting_records.csv")
        
        # Quick performance check
        print("\n📈 RECENT PERFORMANCE (Last 30 days):")
        report = tracker.generate_performance_report(days=30)
        
    else:
        print("   • No bets were updated")

# Run the updater
update_results_standalone()

print("\n🎉 Thank you for using the Enhanced UFC Betting System!")
print("💪 Good luck with your next event!")